In [4]:
import cv2
from PIL import Image, ImageTk
import tkinter as tk
import customtkinter as ctk
import torch
import numpy as np
import vlc

# Charger le modèle YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp5/weights/last.pt',force_reload=True)

# Initialisation de la capture vidéo
cap = cv2.VideoCapture(0)

counter = 0

def reset_counter():
    global counter
    counter = 0
    counterLabel.config(text=str(counter))

def quit_app(event=None):
    global cap
    cap.release()
    app.destroy()

# Création de l'application tkinter
app = ctk.CTk()  # Utilisation de CTk au lieu de Tk pour la compatibilité avec customtkinter
app.geometry("600x600")
app.title("FI9 ALMERDi")

# Cadre pour afficher la vidéo
vid = tk.Label(app)
vid.pack()

# Étiquette pour afficher le compteur
counterLabel = ctk.CTkLabel(app, text=str(counter), height=40, width=120, font=("Arial", 20), text_color="black")
counterLabel.pack(pady=10)

# Bouton pour réinitialiser le compteur
resetButton = ctk.CTkButton(app, text="Reset Counter", command=reset_counter, height=40, width=120, font=("Arial", 20), text_color="white")
resetButton.pack()

# Fonction de détection
def detect():
    global counter
    ret, frame = cap.read()
    if not ret:
        print("Erreur de capture vidéo")
        app.after(10, detect)
        return

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame)
    img = np.squeeze(results.render())
    
    if len(results.xywh[0]) > 0:
        dconf = results.xywh[0][0][4]
        dclass = results.xywh[0][0][5]
        if  dclass == 16.0:  # Assuming dclass == 1.0 is the condition to check
            p = vlc.MediaPlayer(r"C:\Users\elkha\Downloads\videoplayback.wav")
            p.play()
            counter += 1
        
    imgarr = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(image=imgarr)
    vid.configure(image=imgtk)
    vid.image = imgtk
    vid.after(10, detect)
    counterLabel.configure(text=str(counter))

# Lancer la détection après que la fenêtre Tkinter soit prête
app.after(0, detect)

# Lier la touche 'q' à la fonction quit_app
app.bind('<q>', quit_app)

# Boucle principale de l'application
app.mainloop()

# Libérer la capture vidéo si l'application est fermée
cap.release()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\elkha/.cache\torch\hub\master.zip
YOLOv5  2024-7-31 Python-3.12.4 torch-2.3.1+cpu CPU



Exception: [Errno 2] No such file or directory: 'yolov5\\runs\\train\\exp5\\weights\\last.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.